In [34]:
import itertools
import os
import random
import re
import subprocess

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib.patches import Rectangle

# Reading

In [10]:
RUNDIRS = '../logs/rundirs'

# BASENAME_CSV = 'sorted-20241128_210446' 
# BASENAME_CSV = '20241203_133243/sorted' 
BASENAME_CSV = '20241203_170129_all600/sorted' 
I_MAP = 1
BASENAME_CSV_LINEARIZATIONS = '20241204_122017_lin_abc/sorted' 

DIRECTORY_IMAGES = f'images/{BASENAME_CSV.split("/")[0]}'
os.makedirs(DIRECTORY_IMAGES, exist_ok=True)

df_orig = pd.read_csv(f'{RUNDIRS}/{BASENAME_CSV}.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,...,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization
0,20241203_185404,map-generator/generated-maps/2024-11-28_13:19:...,2,AutonomousVehicle,807.09,5,9613.3,5,0,0,...,"(638.2, 471.5)",1548.6,1546.0,1612,NaN,1581.7,NaN,DRIVING,1: [1612],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,20241203_185404,map-generator/generated-maps/2024-11-28_13:19:...,3,AutonomousVehicle,927.04,0,0.0,1,0,0,...,"(737.0, 537.0)",0.0,0.0,1852,ASAP,0.0,NaN,STOP@CP,1: [1852],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,20241203_185404,map-generator/generated-maps/2024-11-28_13:19:...,0,HumanDrivenVehicle,454.31,0,25.8,1,1,1,...,"(749.7, 43.0)",25.8,25.0,454,26,16.2,2.0,STOP@CP,"2: [452, 454]",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,20241203_185404,map-generator/generated-maps/2024-11-28_13:19:...,1,AutonomousVehicle,493.11,8,8813.8,11,0,0,...,"(441.3, 488.1)",937.3,935.0,984,NaN,953.6,NaN,DRIVING,1: [984],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,20241203_204608,map-generator/generated-maps/2024-11-28_13:19:...,2,AutonomousVehicle,479.00,9,8611.8,15,0,0,...,"(487.0, 532.3)",4.7,4.0,956,NaN,925.4,NaN,DRIVING,1: [956],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,20241203_191912,map-generator/generated-maps/2024-11-28_13:17:...,1,AutonomousVehicle,449.08,9,8903.6,12,0,0,...,"(434.2, 402.2)",836.3,834.0,896,NaN,865.5,NaN,DRIVING,1: [896],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2396,20241203_192914,map-generator/generated-maps/2024-11-28_13:17:...,2,AutonomousVehicle,566.43,0,227.9,1,0,0,...,"(555.3, 178.0)",227.9,227.0,1130,ASAP,198.3,NaN,STOP@CP,1: [1130],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2397,20241203_192914,map-generator/generated-maps/2024-11-28_13:17:...,3,AutonomousVehicle,820.88,0,229.6,1,0,0,...,"(677.3, 310.9)",229.6,230.0,1640,ASAP,198.9,NaN,STOP@CP,1: [1640],0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2398,20241203_192914,map-generator/generated-maps/2024-11-28_13:17:...,0,HumanDrivenVehicle,455.64,0,223.4,1,1,1,...,"(555.8, 81.2)",223.4,223.0,455,224,192.7,2.0,STOP@CP,"2: [453, 455]",0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [11]:
def normalize_linearization(lin):
    return tuple(
        np.interp(
            np.linspace(0, 1, 100),
            np.linspace(0, 1, len(lin)), 
            lin
        )
    )


def normalize_df(df_orig):
    df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
    df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
    df_id = pd.concat([
        df_id,
        df_id['filename'].str.extract(r'(?P<dir_map>[^/]+)/(?P<basename_scenario>[^/]+)[.]json$', expand=True),
        df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+)[.]json$', expand=True).astype(int),
        df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
    ], axis=1).rename(columns={'i_locations': 'Positions variant'})
    df_id['filename_screenshot'] = "../map-generator/generated-maps/" + df_id['dir_map'] + '/screenshots/' + df_id['basename_scenario'] + '.png'
    df_id['are_bridges'] = df_id['dir_map'].str.contains('with_bridges')
    df_id['configuration'] = df_id[
        ['i_map', 'are_bridges', 'Positions variant']].agg(
        lambda r: f'map {r['i_map']}, {"with" if r['are_bridges'] else "without"} bridges, pos.var. {r['Positions variant']}', 
        axis=1
    )
    #df_id = df_id[df_id['i_map'] == I_MAP]
    
    df = pd.concat(
        [
            df_id[['i_map', 'are_bridges', 'Positions variant', 'configuration', 
                   'Coordination strategy', 'probabilityForcingForHuman', 'filename_screenshot']],
            df_orig
        ],
        axis=1
    )
    df.sort_values(
        ['i_map', 'are_bridges', 'Positions variant', 'Vehicle ID'], 
        ascending=[True, False, True, True],
        inplace=True,
    )
    
    for col in df.columns:
        if col.startswith('Linearization'):
            series = df[col].apply(lambda x: tuple(map(float, x.split())))
            df[col] = series.apply(normalize_linearization)
            df[f'{col} (non-normalized)'] = series
    
    return df


df_orig_norm = normalize_df(df_orig)
configuration_to_filename_screenshot = {row['configuration']: row['filename_screenshot'] for _, row in df_orig_norm.iterrows()}
df_orig_norm

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,filename_screenshot,Date,Scenario ID,Vehicle ID,...,"traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization,Linearization (non-normalized)
902,1,True,1,"map 1, with bridges, pos.var. 1",stops,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203636,map-generator/generated-maps/2024-11-28_13:17:...,0,...,489.0,487.0,782,NaN,750.4,NaN,DRIVING,"2: [784, 782]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1526,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203436,map-generator/generated-maps/2024-11-28_13:17:...,0,...,486.8,485.0,782,NaN,750.4,NaN,DRIVING,"2: [784, 782]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2094,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203255,map-generator/generated-maps/2024-11-28_13:17:...,0,...,72.1,71.0,784,73,45.4,3.0,STOP@CP,"2: [782, 784]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
903,1,True,1,"map 1, with bridges, pos.var. 1",stops,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203636,map-generator/generated-maps/2024-11-28_13:17:...,1,...,253.5,249.0,254,NaN,224.7,NaN,DRIVING,1: [254],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1527,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203436,map-generator/generated-maps/2024-11-28_13:17:...,1,...,38.9,38.0,254,NaN,224.7,NaN,DRIVING,1: [254],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,10,False,10,"map 10, without bridges, pos.var. 10",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203234,map-generator/generated-maps/2024-11-28_13:19:...,2,...,16.2,16.0,1134,311,279.1,296.8,DRIVING,1: [1134],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1392,10,False,10,"map 10, without bridges, pos.var. 10",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203031,map-generator/generated-maps/2024-11-28_13:19:...,2,...,957.5,956.0,1134,NaN,1102.8,NaN,DRIVING,1: [1134],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
145,10,False,10,"map 10, without bridges, pos.var. 10",stops,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203420,map-generator/generated-maps/2024-11-28_13:19:...,3,...,68.2,67.0,634,NaN,603.4,NaN,DRIVING,1: [634],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
649,10,False,10,"map 10, without bridges, pos.var. 10",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203234,map-generator/generated-maps/2024-11-28_13:19:...,3,...,490.9,489.0,634,NaN,603.6,NaN,DRIVING,1: [634],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [12]:
df_lin_norm = (
    None if BASENAME_CSV_LINEARIZATIONS is None else 
    normalize_df(pd.read_csv(f'{RUNDIRS}/{BASENAME_CSV_LINEARIZATIONS}.csv'))
)
df_lin_norm

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,filename_screenshot,Date,Scenario ID,Vehicle ID,...,"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized)
720,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241204_113441,map-generator/generated-maps/2024-11-28_13:17:...,0,...,45.4,74.3,DRIVING,"2: [782, 784]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
721,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241204_113441,map-generator/generated-maps/2024-11-28_13:17:...,1,...,224.7,NaN,DRIVING,1: [254],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
722,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241204_113441,map-generator/generated-maps/2024-11-28_13:17:...,2,...,401.4,433.9,DRIVING,1: [944],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
723,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241204_113441,map-generator/generated-maps/2024-11-28_13:17:...,3,...,1485.8,NaN,DRIVING,1: [1516],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
760,1,True,2,"map 1, with bridges, pos.var. 2",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241204_113527,map-generator/generated-maps/2024-11-28_13:17:...,0,...,267.9,300.4,DRIVING,"2: [782, 784]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,10,False,9,"map 10, without bridges, pos.var. 9",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241204_113333,map-generator/generated-maps/2024-11-28_13:19:...,3,...,1358.5,NaN,DRIVING,1: [1390],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
696,10,False,10,"map 10, without bridges, pos.var. 10",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241204_113414,map-generator/generated-maps/2024-11-28_13:19:...,0,...,321.2,353.9,DRIVING,"2: [870, 872]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.

In [13]:
def add_linearizations(df_orig_norm, df_lin_norm):
    if df_lin_norm is None:
        return df_orig_norm

    # Get columns that start with 'Linearization'
    linearization_cols = [col for col in df_lin_norm.columns if col.startswith('Linearization')]
    assert linearization_cols
    
    # Specified keys for merging
    keys = ['i_map', 'are_bridges', 'Positions variant', 'Vehicle ID']
    
    # Check if keys exist in both dataframes
    missing_keys = [key for key in keys
                    if key not in df_lin_norm.columns
                    or key not in df_orig_norm.columns]
    if missing_keys:
        raise KeyError(f"Missing key columns in either DataFrame: {missing_keys}")
    
    # Perform an inner merge to ensure all data from df_lin is matched and not missing
    merged_df = pd.merge(df_orig_norm, df_lin_norm[keys + linearization_cols], on=keys, how='inner')
    
    # Check if the merged DataFrame has any missing data from df_lin
    if merged_df[linearization_cols].isnull().any().any():
        raise ValueError("Some linearization is missing in the merged DataFrame.")
    
    return merged_df


df_all = add_linearizations(df_orig_norm, df_lin_norm)
df_all.to_csv('data/df_all.csv', index=False)
df_all

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,filename_screenshot,Date,Scenario ID,Vehicle ID,...,status,future missions,Linearization,Linearization (non-normalized),Linearization A,Linearization B,Linearization C,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized)
0,1,True,1,"map 1, with bridges, pos.var. 1",stops,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203636,map-generator/generated-maps/2024-11-28_13:17:...,0,...,DRIVING,"2: [784, 782]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203436,map-generator/generated-maps/2024-11-28_13:17:...,0,...,DRIVING,"2: [784, 782]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203255,map-generator/generated-maps/2024-11-28_13:17:...,0,...,STOP@CP,"2: [782, 784]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,True,1,"map 1, with bridges, pos.var. 1",stops,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203636,map-generator/generated-maps/2024-11-28_13:17:...,1,...,DRIVING,1: [254],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203436,map-generator/generated-maps/2024-11-28_13:17:...,1,...,DRIVING,1: [254],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,10,False,10,"map 10, without bridges, pos.var. 10",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203234,map-generator/generated-maps/2024-11-28_13:19:...,2,...,DRIVING,1: [1134],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 

# Filtering by `i_map`

In [14]:
df_map = df_all[df_all['i_map'] == I_MAP]
df_map

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,filename_screenshot,Date,Scenario ID,Vehicle ID,...,status,future missions,Linearization,Linearization (non-normalized),Linearization A,Linearization B,Linearization C,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized)
0,1,True,1,"map 1, with bridges, pos.var. 1",stops,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203636,map-generator/generated-maps/2024-11-28_13:17:...,0,...,DRIVING,"2: [784, 782]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203436,map-generator/generated-maps/2024-11-28_13:17:...,0,...,DRIVING,"2: [784, 782]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203255,map-generator/generated-maps/2024-11-28_13:17:...,0,...,STOP@CP,"2: [782, 784]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,True,1,"map 1, with bridges, pos.var. 1",stops,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203636,map-generator/generated-maps/2024-11-28_13:17:...,1,...,DRIVING,1: [254],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_203436,map-generator/generated-maps/2024-11-28_13:17:...,1,...,DRIVING,1: [254],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1,False,10,"map 1, without bridges, pos.var. 10",stops,1.0,../map-generator/generated-maps/2024-11-28_13:...,20241203_205337,map-generator/generated-maps/2024-11-28_13:19:...,2,...,DRIVING,1: [734],"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [15]:
series_blocks = df_map[df_map['Vehicle type'] != 'HumanDrivenVehicle'].groupby(['configuration', 'are_bridges'], sort=False)['Is blocked'].sum()
index_blocked = series_blocks[series_blocks != 0].index
index_nonblocked = series_blocks[series_blocks == 0].index
index_nonblocked[~index_nonblocked.get_level_values('are_bridges')].get_level_values('configuration')

Index(['map 1, without bridges, pos.var. 1',
       'map 1, without bridges, pos.var. 2',
       'map 1, without bridges, pos.var. 3',
       'map 1, without bridges, pos.var. 4',
       'map 1, without bridges, pos.var. 5',
       'map 1, without bridges, pos.var. 6',
       'map 1, without bridges, pos.var. 7',
       'map 1, without bridges, pos.var. 8',
       'map 1, without bridges, pos.var. 9',
       'map 1, without bridges, pos.var. 10'],
      dtype='object', name='configuration')

# Main plots

In [35]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_IMAGES}/{basename}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    
    plt.close(fig)
    
    return filename


def same_value(series):
    assert series.nunique() == 1, series
    return series.iloc[0]
    

def make_misbehaviors(df_nonbaseline):
    misbehaviors = []
    
    probabilityForcingForHuman = same_value(df_nonbaseline['probabilityForcingForHuman'])
    if probabilityForcingForHuman > 0.0:
        misbehaviors.append(f'violation of priorities ({"random" if probabilityForcingForHuman < 1.0 else "constant"})')
        
    isCanPassFirstActive = same_value(df_nonbaseline['isCanPassFirstActive'])
    if isCanPassFirstActive.startswith('hum=true, '):
        misbehaviors.append('can pass first')
    elif isCanPassFirstActive.startswith('hum=false, '):
        pass
    else:
        raise ValueError(isCanPassFirstActive)
    
    if 'probabilitySlowingDownForHuman' in df_nonbaseline.columns:
        probabilitySlowingDownForHuman = float(same_value(df_nonbaseline['probabilitySlowingDownForHuman']))
        if probabilitySlowingDownForHuman > 0.0:
            misbehaviors.append(f'moving slowly ({"random" if probabilitySlowingDownForHuman < 1.0 else "constant"})')
    
    return misbehaviors


def make_subplots(ncols):
    return plt.subplots(1, ncols, figsize=(20, 6), sharey=True, squeeze=False)
    

def plot_title(df, *, title2):
    positions = df.index.get_level_values('Positions variant').unique()
    fig, axes = make_subplots(len(positions))
    default_fig_width, default_fig_height = fig.get_size_inches()
    plt.close(fig)
    
    fig = plt.figure(figsize=(default_fig_width, 1))
    
    misbehaviors = make_misbehaviors(df[df['Coordination strategy'] != 'baseline'])
    title3 = 'Human (mis)behaviour actions: ' + ('none' if not misbehaviors else ', '.join(misbehaviors))    
    
    title = f'{title2}\n{title3}'
    fig.suptitle(title, fontsize=16)
    
    fig.tight_layout()
    title1 = '.title'
    filename_png = save_and_show(fig, f'{title1}: {title2}')
    
    return filename_png
    
    
class Formula:
    def __init__(self, label, expression=None):
        self.label = label
        self.expression = expression if expression is not None else label
        
    def __str__(self):
        return self.label.replace('`', '')
    
    def __repr__(self):
        return f'<Formula: {self.label!r}>'
    
    def apply(self, df):
        if isinstance(self.expression, str):
            return df.eval(self.expression)
        return df.apply(self.expression, axis=1)
    
    
def plot_vertical_heatmap(ax, max_length, df, column):    
    matrix = df[column]
    
    is_normalized = not column.endswith(' (non-normalized)')

    # Expand each list to have the same length by interpolating values
    expanded_data = []
    for row in matrix:
        if is_normalized:
            expanded_data.append(row)
        else:
            expanded_data.append(row + (np.nan,) * (max_length - len(row)))

    # Convert the expanded data to a 2D NumPy array
    data_array = np.array(expanded_data)

    # Transpose the data array to have heatmaps side-by-side (columns represent each row)
    data_array_transposed = data_array.T

    # Plot the heatmap - each column is a vertical slice now (grayscale, 0 is white, max is black)
    im = ax.imshow(data_array_transposed, aspect='auto', cmap='gray_r', interpolation='nearest')

    # Set labels for better readability
    ax.set_xlabel('Vehicle ID')
    ax.set_xticks(np.arange(len(df.index)))
    ax.set_xticklabels(df.index.get_level_values("Vehicle ID"))
    
    if is_normalized:
        ax.set_ylabel('Stage along the path')
        y_ticks = np.linspace(0, max_length - 1, 5)  # Set 5 evenly spaced ticks from 0 to max_length - 1
        ax.set_yticks(y_ticks)  # Apply these y-tick positions
        ax.set_yticklabels([f'{int((tick / (max_length - 1)) * 100)}%' for tick in y_ticks])  # Label ticks from 0% to 100%
    else:    
        ax.set_ylabel('Meters of the path')
        
        # Generate mask for NaN values
        nan_mask = np.isnan(data_array_transposed)

        """
        # Adding diagonal stripes for NaN regions
        for col in range(data_array_transposed.shape[1]):
            nan_rows = np.where(nan_mask[:, col])[0]
            if len(nan_rows) > 0:
                start_row = nan_rows[0]
                height = data_array_transposed.shape[0] - start_row
                rect = Rectangle((col - 0.5, start_row - 0.5), 1, height, hatch='///', edgecolor='black', facecolor='none', linewidth=0)
                ax.add_patch(rect)
        """
        
        # Adding rectangles around non-NaN regions for each column
        for col in range(data_array_transposed.shape[1]):
            nan_rows = np.where(nan_mask[:, col])[0]
            
            # Determine the start and end of non-NaN regions
            if len(nan_rows) == 0:
                # No NaN values in the column; entire column is non-NaN
                start_row = 0
                end_row = data_array_transposed.shape[0]
            else:
                # If there are NaNs, find the first occurrence
                start_row = 0
                end_row = nan_rows[0]
            
            # Draw rectangle around non-NaN values
            rect = Rectangle((col - 0.5, start_row - 0.5), 1, end_row - start_row, edgecolor='black', facecolor='none', linewidth=2)
            ax.add_patch(rect)

        # Setting labels and showing the plot
        #plt.colorbar(im, ax=ax)
    
    # Return the image object for colorbar usage
    return im


def plot_lin(df, *, title2, column):
    #IPython.display.display(df)
    
    # Plot configuration
    positions = df.index.get_level_values('Positions variant').unique()    
    fig, axes = make_subplots(len(positions))
    axes: list[list[Axes]]

    # Find the maximum length of rows
    max_length = max(
        max(df.loc[position, column].apply(len))
        for position in positions
    )
    
    images = []
    for i, position in enumerate(positions):
        ax: Axes = axes[0][i]
        df_pos = df.loc[position]
        images.append(plot_vertical_heatmap(ax, max_length, df_pos, column=column))
        ax.set_title(f'Position {position}')
    
    title1 = f'Paths segmentation based on CS density ({column})'
    column2description = {
        'Linearization': 'From the full simulation. Same as Linearization A.', 
        'Linearization A': 'From the short simulation. CS count: +1 for each CS.', 
        'Linearization B': "From the short simulation. For each CS: +1 / other's path length.", 
        'Linearization C': "From the short simulation. For each CS: + other's CS length / other's path length.", 
    }
    description = column2description.get(column)
    
    title = title1
    if description is not None:
        title += f'\n({description})'
    fig.suptitle(title, fontsize=16)
    
    fig.tight_layout()
    fig.subplots_adjust(top=0.85)
    
    cax = fig.add_axes([0.85, 0.9, 0.1, 0.03])  # Adjust values to position the colorbar correctly
    fig.colorbar(
        images[0], cax=cax, label='Density',
        orientation='horizontal', location='top',
    )
    
    filename_png = save_and_show(fig, f'{title1}: {title2}')
    
    #print(id(df))
    #IPython.display.display(df)
    return filename_png
    
    
def plot_aut_hum(df, *, title2, dfs_y1, dfs_y2, mode):
    #IPython.display.display(df)
    
    # Plot configuration
    positions = df.index.get_level_values('Positions variant').unique()    
    fig, axes = make_subplots(len(positions))
    axes: list[list[Axes]]
    bar_width = 0.4
    
    strategies = df.index.get_level_values('Coordination strategy').unique()    
    strategy2label = {'baseline': 'baseline\n(no human effect)', 'stops': 'stops (local)'}        

    formulas_y2_aut = [Formula('`No. of collisions`'), Formula('`No. of near-misses`')]
    colors_y2_aut = ['red', 'yellow']
    
    formulas_y2_hum = [Formula('`No. of violations`')] + formulas_y2_aut
    colors_y2_hum = ['black'] + colors_y2_aut
    
    formulas_y2_cmp = [Formula(f'`{col}` / `No. of violations`', 
                               lambda row, col=col: row[col] / row['No. of violations'] if row['No. of violations'] > 0 else 0)
                       for col in ('No. of collisions', 'No. of near-misses')]
    colors_y2_cmp = ['red', 'yellow']
    
    if mode in ('aut', 'hum'):
        column_y1 = 'Total distance traveled (m)'
        color_y1 = 'tab:blue'
        
        if mode == 'aut':
            formulas_y2 = formulas_y2_aut
            colors_y2 = colors_y2_aut
            id_vehicle_max = df['Vehicle ID'].max()
            title1 = f'Automated vehicles (summarised for AV1-AV{id_vehicle_max})' 
        else:
            formulas_y2 = formulas_y2_hum
            colors_y2 = colors_y2_hum
            title1 = 'Human-driven vehicle'
            
    elif mode == 'cmp':
        column_y1 = None
        color_y1 = None
        
        formulas_y2 = formulas_y2_cmp
        colors_y2 = colors_y2_cmp
        title1 = 'Collisions rate'
        
    else:
        raise ValueError(mode)
    
    # Get the global max values for consistent y-axis scaling
    if column_y1 is not None:
        y1_max = max(dfx[column_y1].max() for dfx in dfs_y1)
        y1_lim = y1_max * 1.1
    y2_maxes = [formula.apply(dfx).max()
                for dfx in dfs_y2
                for formula in (formulas_y2 if mode == 'cmp' else set(formulas_y2_aut + formulas_y2_hum))]
    y2_max = max(y2_maxes)
    y2_lim = y2_max * 1.1
    if mode == 'cmp':
        y2_lim = 2.0
    
    # Iterate through each Positions variant
    for i, position in enumerate(positions):
        ax: Axes = axes[0][i]
        df_pos = df.loc[position]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars
        handles = []
        if column_y1 is None:
            ax.set_yticks([])
        else:
            handles += [ax.bar(x_positions, df_pos[column_y1], width=bar_width, label=column_y1, color=color_y1)[0]]
            ax.set_xlabel('Coordination Strategy')
            ax.set_ylabel(column_y1, color=color_y1)
            ax.tick_params(axis='y', labelcolor=color_y1)
            ax.set_ylim(0, y1_lim)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points
        handles += [ax_right.plot(x_positions, formula.apply(df_pos), label=str(formula),
                                  marker='o', linestyle='', color=color)[0]
                    for formula, color in zip(formulas_y2, colors_y2)]
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_lim)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels([strategy2label.get(s, s) for s in strategies], rotation=60, ha='right')
        ax.set_title(f'Position {position}')
        ax.grid(axis='y')
        
    fig.suptitle(title1, fontsize=16)
    
    labels = [str(x) for x in [column_y1, *formulas_y2] if x is not None]
    fig.legend(handles=handles, labels=labels, ncol=len(handles), loc='upper right')
    
    fig.tight_layout()
    filename_png = save_and_show(fig, f'{title1}: {title2}')
    
    #print(id(df))
    #IPython.display.display(df)    
    return filename_png


def plot_all(df_map):
    key2df = {}
    for are_bridges in True, False:
        for is_aut in None, True, False:
            dfx = df_map[df_map['are_bridges'] == are_bridges]
            dfx = dfx[dfx['configuration'].isin(index_nonblocked.get_level_values('configuration'))]
            if is_aut is None:
                dfx = dfx.groupby(['Positions variant', 'Vehicle ID']).agg({
                    **{col: same_value for col in dfx.columns if col.startswith('Linearization')},
                })
            else:
                dfx = dfx[dfx['Vehicle type'] == ('AutonomousVehicle' if is_aut else 'HumanDrivenVehicle')]
                dfx = dfx.groupby(['Positions variant', 'Coordination strategy']).agg({
                    **{col: 'sum' for col in dfx.columns},
                    **{col: same_value for col in (
                        'Positions variant', 'Coordination strategy', 'configuration',
                        'probabilityForcingForHuman', 'isCanPassFirstActive', 'probabilitySlowingDownForHuman',
                    ) if col in dfx.columns},
                    **{col: 'max' for col in ('Vehicle ID', )},
                })
                
            key2df[are_bridges, is_aut] = dfx          
            
    for are_bridges in True, False:
        filenames_png = [
            plot_title(
                key2df[are_bridges, True],
                title2=f"Map {I_MAP} ({'with' if are_bridges else 'without'} bridges)",
            )
        ]
        cols_lin = [col 
                    for col in df_map 
                    if re.match(r'Linearization([^(]*| C \(.*)$', col)]
        for mode in *cols_lin, 'aut', 'hum', 'cmp':
            is_aut = mode == 'aut'
            title2 = f"Map {I_MAP} ({'with' if are_bridges else 'without'} bridges)"
            
            filenames_png.append(
                plot_lin(
                    key2df[are_bridges, None],
                    title2=title2,
                    column=mode,
                )
                if mode.startswith('Linearization') else
                plot_aut_hum(
                    key2df[are_bridges, is_aut],
                    title2=title2, 
                    dfs_y1=[dfx for (_, is_aut_dfx), dfx in key2df.items() if is_aut_dfx == is_aut],
                    dfs_y2=[key2df[are_bridges, is_aut_dfx] for is_aut_dfx in ([is_aut] if mode == 'cmp' else [True, False])], 
                    mode=mode,
                )
            )
            
        #IPython.display.display(dfx)
        filename_out_png = f'{DIRECTORY_IMAGES}/All: Map {I_MAP} (' + ('with' if are_bridges else 'without') + ' bridges).png'
        subprocess.run(['convert', *filenames_png, '-append', filename_out_png], check=True)
        print()
        

plot_all(df_map)

# Maps

In [36]:
def show_maps(title, configurations, ncols): 
    nrows = max(1, (len(configurations) + ncols - 1) // ncols)
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    #print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(configurations)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, configuration in zip(axes, configurations):
        filename_screenshot = configuration_to_filename_screenshot[configuration]
        try:
            image = plt.imread(filename_screenshot)
            ax.imshow(image)
        except Exception as exc:
            print(exc)
        ax.title.set_text(f'Configuration:\n{configuration}')
    
    fig.suptitle(title, fontsize=16)    
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.1, hspace=0.3)
    save_and_show(fig, title)
    
    
for title, index in {'Non-blocked': index_nonblocked, 'Blocked': index_blocked}.items():
    for are_bridges in True, False:
        show_maps(f'{title}: Map {I_MAP} ({"with" if are_bridges else "without"} bridges)', 
                  index[index.get_level_values('are_bridges') == are_bridges].get_level_values('configuration'), 
                  4)